In [170]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime

from config import gkey

In [171]:
file = 'output_data/cities.csv'
weather_df = pd.read_csv(file)
weather_df.head()

,city,latitude,longitude,max temperature,humidity,cloudiness,wind speed,country,date
0,rocha,-34.4833,-54.3333,71.19,76,93,7.43,UY,1658948126
1,bredasdorp,-34.5322,20.0403,51.01,91,0,6.96,ZA,1658948315
2,khanpur,28.6453,70.6567,90.54,64,100,8.93,PK,1658948291
3,rikitea,-23.1203,-134.9692,73.90,74,100,13.98,PF,1658948316
4,perez,-32.9983,-60.7679,66.09,82,100,11.50,AR,1658948316


### Humidity Heatmap

In [237]:
cities = weather_df[["latitude", "longitude"]]

humidity = weather_df['humidity']
humidity.max()

100

In [243]:
# Plot Heatmap
figure_layout = {
    'width': '700px',
    'height': '500px'
}

fig = gmaps.figure(layout=figure_layout,zoom_level = 1, center = (40.866667, 34.566667))

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='500px', width='700px'))

### Vacation Spots

Criteria: 
* Temp between 65 and 85 degrees
* Wind speed below 5 mph
* 0% cloudiness

In [244]:
vacation_spots = weather_df.loc[
    ((weather_df["max temperature"]>=65) & (weather_df["max temperature"]<85)) &
    (weather_df['wind speed']<5)&
    (weather_df["cloudiness"]==0)
]
vacation_spots

,city,latitude,longitude,max temperature,humidity,cloudiness,wind speed,country,date
30,ardabil,38.2498,48.2933,66.18,82,0,1.39,IR,1658948741
91,gucheng,35.9115,111.3192,70.56,76,0,3.42,CN,1658948830
185,terrace,54.5163,-128.6035,84.67,50,0,0.96,CA,1658948973
231,farafangana,-22.8167,47.8333,67.48,80,0,2.21,MG,1658949042
279,prince george,53.9166,-122.7530,84.25,52,0,1.99,CA,1658949118
282,andrelandia,-21.7397,-44.3092,75.43,34,0,2.55,BR,1658949122
292,saint-francois,46.4154,3.9054,78.24,34,0,4.61,FR,1658948939
306,korzhevskiy,45.1941,37.7195,66.65,73,0,3.09,RU,1658949160
311,borogontsy,62.6683,131.1683,65.19,72,0,2.30,RU,1658949167
343,vlakhopoulon,37.0333,21.7833,81.52,71,0,1.97,GR,1658949219


In [155]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []
city = []
country = []
for index,row in vacation_spots.iterrows():
    my_params = {
    "location": f"{row['latitude']},{row['longitude']}",
    "radius": 5000,
    "keyword": "hotel",
    "key": gkey
    }
    response = requests.get(base_url,params=my_params).json()
    try:
        hotels.append(response['results'][0]['name'])
        city.append(row["city"])
        country.append(row["country"])
    except:
        # hotel probably not within specified distance
        pass
    time.sleep(1)

In [245]:
hotels_df = pd.DataFrame({
    "hotel name":hotels,
    "city":city,
    "country":country
})
hotel_dict = hotels_df.to_dict('records')
full_df = pd.merge(hotels_df, weather_df, how = "left", on=["city","country"])

In [246]:
cities = full_df[["latitude", "longitude"]]
fig = gmaps.figure(layout=figure_layout)

In [247]:
# many thanks to this documentation for helping me figure this part out!
# https://buildmedia.readthedocs.org/media/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotel_dict]

In [248]:
markers = gmaps.marker_layer(cities, info_box_content=hotel_info)                            

fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='500px', width='700px'))